# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: build: File exists
-- The C compiler identification is AppleClang 10.0.1.10010046
-- The CXX compiler identification is AppleClang 10.0.1.10010046
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /Library/Developer/CommandLineTools/usr/bin/c++
-- Check for working CXX compiler: /Library/Developer/CommandLineTools/usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /anaconda3/bin/python (found version "3.7.3") 
-- Found PythonLibs: /anaconda3/lib/libpython3.7m.dylib
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Tes

In [2]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import sentiment


/Users/vpomsztein/Documents/PrivateRepositories/SentimentAnalysisMetNum/notebooks
Python 3.7.3


In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

In [ ]:
df.describe()

In [ ]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]

text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))



In [ ]:
from __future__ import division
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum() / label_train.shape[0], 
    (label_train == 'neg').sum() / label_train.shape[0]
))


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)

vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

In [76]:
import numpy as np
from numpy import linalg as LA

matrix = np.matrix([[1, 0, 1],[0, 2, 1],[0, 0, 3]])
w, v = sentiment.get_first_eigenvalues(matrix, 3, num_iter=50000)
print(w)
print(v)

print('-------------')

w, v = LA.eig(matrix)
print(w)
print(v)

[3. 2. 1.]
[[-0.33333333  0.40824829 -0.57735027]
 [-0.66666667 -0.40824829  0.57735027]
 [-0.66666667  0.81649658  0.57735027]]
[[-0.9106836 ]
 [ 0.23914631]
 [-1.73205081]]
[[-1.        ]
 [ 1.22474487]
 [-1.73205081]]
[[-0.0893164 ]
 [-0.23914631]
 [ 1.73205081]]
[[-1.33333333]
 [-0.81649658]
 [ 1.15470054]]
[[-0.0893164 ]
 [ 2.21034343]
 [ 1.73205081]]
[[-0.66666667]
 [ 0.81649658]
 [ 0.57735027]]
-------------
[1. 2. 3.]
[[1.         0.         0.33333333]
 [0.         1.         0.66666667]
 [0.         0.         0.66666667]]


In [ ]:
import sentiment

# Usar PCA acá y pasarle el X_train modificado
pca = sentiment.PCA(2)
pca.fit(X_train[:2])
pca_x_train = pca.transform(X_train[:2])
pca_y_train = pca.transform(y_train[:2])

pca_x_test = pca.transform(X_test[:2])
pca_y_test = pca.transform(y_test[:2])

clf = sentiment.KNNClassifier(1000)

clf.fit(X_train, y_train)


In [ ]:
%%time
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test[:2])

acc = accuracy_score(y_test[:2], y_pred[:2])
print("Accuracy: {}".format(acc))